In [ ]:
"""
Purpose: To explore the cells that have
already been labeled as excitatory and inhibitory


"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import sys
sys.path.append("/meshAfterParty/")

In [3]:
from importlib import reload

import neuron_utils as nru

import neuron

import neuron_visualizations as nviz

import time

import datajoint_utils as du

import numpy as np

import proofreading_utils as pru

import preprocessing_vp2 as pre

# -- For the axon classification --

import neuron_searching as ns

import skeleton_utils as sk

import numpy_utils as nu

import networkx_utils as xu

INFO - 2021-01-22 07:35:15,134 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-22 07:35:15,135 - settings - Setting database.user to celiib
INFO - 2021-01-22 07:35:15,136 - settings - Setting database.password to newceliipass
INFO - 2021-01-22 07:35:15,139 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-01-22 07:35:15,140 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-22 07:35:15,150 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-01-22 07:35:15,444 - settings - Setting enable_python_native_blobs to True


In [4]:
minnie,_ = du.configure_minnie_vm()

INFO - 2021-01-22 07:35:15,529 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-22 07:35:15,809 - settings - Setting enable_python_native_blobs to True


# The tables for inhibitory, excitatory cells

In [5]:
classified_table = minnie.BaylorManualCellType() & 'nucleus_version=3'
classified_table

nucleus_id id of segmented nucleus,nucleus_version segmentation version,segment_id segment_id merged to nucleus,"cell_type morphology (valid options: 'excitatory', 'inhibitory', 'other', 'unknown')",classifier name of human classifier,timestamp
43915,3,864691135272175377,unknown,stelios,2020-11-27 03:35:00
43989,3,864691135781479503,excitatory,stelios,2020-11-27 03:30:20
71096,3,864691135269778085,excitatory,stelios,2020-11-27 03:36:45
73494,3,864691135776583776,excitatory,stelios,2020-11-27 03:31:02
76763,3,864691136333770163,other,stelios,2020-11-27 02:22:03
81095,3,864691135462264733,other,stelios,2020-11-27 02:21:50
81844,3,864691135570922063,unknown,stelios,2020-11-27 03:33:13
85823,3,864691135859328688,other,stelios,2020-11-27 03:35:47
106594,3,864691135617716623,other,stelios,2020-11-27 02:21:38
112738,3,864691136436394654,excitatory,stelios,2020-11-27 04:29:27


In [6]:
# looking at how many of these neurons are in decimation already
(minnie.Decimation() & dict(decimation_ratio=0.25)) & classified_table.proj()

segment_id segment id unique within each Segmentation,version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),n_vertices,n_faces,mesh in-place path to the hdf5 (decimated) mesh file
864691135272175377,0,0.25,49415,116541,=BLOB=
864691135781479503,0,0.25,49896,99336,=BLOB=
864691135269778085,0,0.25,54886,137249,=BLOB=
864691135776583776,0,0.25,38756,93834,=BLOB=
864691136333770163,0,0.25,443884,914559,=BLOB=
864691135462264733,0,0.25,383607,786834,=BLOB=
864691135570922063,0,0.25,12729,25588,=BLOB=
864691135859328688,0,0.25,15508,31041,=BLOB=
864691135617716623,0,0.25,1429227,2911995,=BLOB=
864691136436394654,0,0.25,414218,847287,=BLOB=


In [7]:
import datajoint as dj
dj.U("segment_id") & (minnie.BaylorSegmentCentroid()  & classified_table.proj())

segment_id segment id unique within each Segmentation
864691135272175377
864691135781479503
864691135269778085
864691135776583776
864691136333770163
864691135462264733
864691135570922063
864691135859328688
864691135617716623
864691136436394654


In [8]:
classified_decomp = dj.U("segment_id") & (minnie.Decomposition()  & classified_table.proj())
classified_decomp

segment_id segment id unique within each Segmentation
864691135272175377
864691135781479503
864691135269778085
864691135570922063
864691135859328688
864691136004721098
864691135523678660
864691135382160370
864691135725629503
864691135337703398


In [9]:
excitatory_neurons = minnie.Decomposition & (classified_table & dict(cell_type='excitatory'))
excitatory_neurons

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691135781479503,0,0.25,30.00,1,0,1,=BLOB=,49896,99336,0,0,0,0,1,5,17,5,226672.13053857177,75504.12284558205,10157.276,382.387,377.228,625.586,625.586,41,0.00018087799281977992,2.411764705882353,226672.131,17,0.00018087799245157315,2.412,8161926611.552537,118414501.81582816,36007.63178146269,36007.631708163266,480113330.0913257,161.4672
864691135269778085,0,0.25,30.00,0,0,1,=BLOB=,54886,137249,0,0,0,0,1,1,9,9,259616.22842272613,259616.22842272613,31382.993,375.898,337.387,532.281,497.674,126,0.000485331755897931,14.0,259616.228,9,0.0004853317566881836,14.0,21692088970.324295,130208319.59808476,83554.44150048913,83554.44163653858,2410232107.8138103,121.7504
864691136004721098,0,0.25,30.00,1,0,1,=BLOB=,104080,224182,0,0,0,0,1,6,26,11,1002176.4388136197,448367.4893572848,29443.614,265.921,247.11,418.725,402.884,297,0.00029635500147218565,11.423076923076923,941501.121,24,0.00031545368707001253,12.375,72894254146.9965,129506109.23066711,72735.94880487213,77423.43850804241,3037260589.4581876,321.2925
864691135725629503,0,0.25,30.00,0,0,1,=BLOB=,294527,605009,0,0,0,0,1,7,122,90,3983774.4188000374,3075868.689500789,16642.287,72.56,72.56,302.027,278.202,354,8.88604531244089e-05,2.901639344262295,1951991.164,29,0.0001813532799372856,12.207,40042327195.412094,57745238.84199929,10051.35406423774,20513.580150310605,1380769903.2900722,1061.656
864691135337703398,0,0.25,30.00,0,0,1,=BLOB=,315689,646904,0,0,0,0,1,6,195,132,5107623.215336702,3433465.2233310975,17308.591,74.907,74.907,286.098,267.271,521,0.00010200439187361923,2.6717948717948716,1342610.209,33,0.00038805008073642614,15.788,119660747349.65529,173817254.91832542,23427.872868607257,89125.45618045072,3626083253.0198574,21460.8731
864691135581598573,0,0.25,30.00,0,0,1,=BLOB=,260826,539242,0,0,0,0,1,6,97,76,3135944.7738076993,2357792.352163731,25575.906,75.863,75.863,415.919,385.193,632,0.000201534161340

In [10]:
inhibitory_neurons = minnie.Decomposition & (classified_table & dict(cell_type='inhibitory'))
inhibitory_neurons

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691135523678660,0,0.25,30.00,0,0,1,=BLOB=,303484,604221,0,0,0,0,1,7,101,71,3942156.6847181707,2832232.158793305,20981.081,87.35,87.35,448.591,435.089,524,0.00013292216466973366,5.188118811881188,1977146.733,40,0.00026502838219038746,13.1,91867403514.80359,113411558.9594806,23303.8437743809,46464.63612511434,2296685087.8700895,945.1412
864691134884755962,0,0.25,30.00,0,0,1,=BLOB=,974236,1972411,0,0,0,0,1,6,506,433,15259377.389223402,12144163.476963362,21315.999,103.155,103.155,330.533,329.54,733,4.803603589473218e-05,1.448616600790514,6096199.955,184,0.00012023883819604798,3.984,93290513367.66779,40734343.56715155,6113.651362574737,15303.05994821454,507013659.60689014,5584.3764
864691135181813250,0,0.25,30.00,0,0,1,=BLOB=,367143,736669,0,1,1,0,1,13,341,67,8601754.353709135,1592600.2628614653,16913.445,72.507,72.507,119.729,119.729,14,1.6275749602129832e-06,0.04105571847507331,366681.345,21,3.8180289755400565e-05,0.667,3578468037.248188,101376868.89817174,416.01606952483223,9759.067610183954,170403239.86896133,611.0495
864691135348317426,0,0.25,30.00,0,0,1,=BLOB=,244198,487468,0,0,0,0,1,10,64,23,2328781.4672830035,760834.6556884258,16454.513,217.548,217.548,432.842,430.804,303,0.00013011096328996083,4.734375,2008227.523,42,0.0001508793184685379,7.214,55057316356.31905,92279514.65500386,23642.113753401944,27415.875803789117,1310888484.6742632,784.7755
864691135500208836,0,0.25,30.00,0,0,1,=BLOB=,283828,565772,0,0,0,0,1,3,169,140,5533640.1423449,4579212.255149077,23109.866,75.784,75.784,224.012,222.812,270,4.8792475306423976e-05,1.5976331360946745,1017098.678,25,0.00026546096838010047,10.8,35743148103.013695,58544318.92308088,6459.246930333891,35142.261882886545,1429725924.1205478,802.9508
864691135937157636,0,0.25,30.00,1,0,1,=BLOB=,105603,222884,0,1,1,0,1,5,46,24,1343016.9446939288,763308.5726518938,22759.187,231.717,231.717,398.179,398.179,97,7.22

In [ ]:
classified_table & dict(segment_id=864691135269732773)

# visualizing the inhibitory neurons

In [11]:
#visualizing inhibitory neurons
inhib_segs = inhibitory_neurons.fetch("segment_id")
inhib_segment_id = inhib_segs[-22]
inhib_segment_id = 864691135644605167
inhib_segment_id

864691135644605167

In [ ]:
neuron = reload(neuron)
inh_obj = (minnie.Decomposition & dict(segment_id=inhib_segment_id)).fetch1("decomposition")
inh_obj.calculate_spines()

In [ ]:
inh_obj.segment_id

In [ ]:
nviz.plot_spines(inh_obj)

In [ ]:
ret_col = nviz.visualize_neuron(inh_obj,
                     limb_branch_dict=dict(L0="all"),
                     return_color_dict=True)

In [ ]:
nviz.plot_limb_concept_network_2D(inh_obj,
                                 node_colors=ret_col)

In [ ]:
neuron_obj = inh_obj

neuron_obj.spine_density_eligible

In [ ]:
curr_branch = neuron_obj[0][24]
print(f"spine_density = {curr_branch.spine_density}, n_spines = {curr_branch.n_spines},"
      f"\n median_mesh_center = {curr_branch.width_new['median_mesh_center']},no_spine_median_mesh_center = {curr_branch.width_new['no_spine_median_mesh_center']}")
nviz.plot_branches_with_spines(curr_branch)

In [ ]:
import neuron_searching as ns
axon_like_limb_branch_dict = ns.query_neuron(inh_obj,
               functions_list=["n_spines","median_mesh_center"],
               query = "(median_mesh_center < 140) and (n_spines < 2)")
axon_like_limb_branch_dict

In [ ]:
nviz.visualize_neuron(inh_obj,
                     visualize_type=["mesh"],
                     limb_branch_dict=axon_like_limb_branch_dict,
                     mesh_color="red",
                     mesh_whole_neuron=True)

# The Brainstorming on Classification

In [ ]:
"""
Have 3 Groups: 1) Sparsely spined, 2) Densely spined 3) No spine

For the sparsely spined group: 
1) Look for axon (Excitatory)
2) Look for long apical up (Excitatory)

--> then can also error out some axon like parts




"""

In [ ]:
len(minnie.Decomposition())

# Finding all 4 crossovers

In [ ]:
nviz.visualize_neuron(inh_obj,
                      visualize_type=["mesh","skeleton"],
                      limb_branch_dict=dict(L0="all"),
                      mesh_color="green",
                      skeleton_color="blue",)

In [ ]:
nviz.plot_objects(meshes_colors=)

In [41]:
len(minnie.Decomposition())

49742

In [48]:
du.plot_decimated_mesh_with_somas(864691135428521008,
                                  main_mesh_color=[1,0,0,0.8],)

Segment_id = 864691135428521008
vertices = 328724, faces= = 662582
No glia to plot
subtracting glia and nuclei
curr_colors = ['red']
curr_soma_meshes = [<trimesh.Trimesh(vertices.shape=(8126, 3), faces.shape=(15886, 3))>]
other_meshes_to_plot = []


In [ ]:
clu.axon_classification(inh_obj)

# Viewing the excitatoy neuons

In [38]:
#visualizing inhibitory neurons
excitatory_segs = excitatory_neurons.fetch("segment_id")
excitatory_seg = excitatory_segs[-75]

In [39]:
du.plot_decimated_mesh_with_somas(excitatory_seg)

Segment_id = 864691135657796994
vertices = 387661, faces= = 791439
No glia to plot
subtracting glia and nuclei
curr_colors = ['red']
curr_soma_meshes = [<trimesh.Trimesh(vertices.shape=(22051, 3), faces.shape=(43349, 3))>]
other_meshes_to_plot = []


In [13]:
exc_obj = (minnie.Decomposition & dict(segment_id=excitatory_seg)).fetch1("decomposition")

Decompressing Neuron in minimal output mode...please wait


In [14]:
exc_obj.calculate_spines()

query = median_mesh_center > 115 and n_faces_branch>100
smoothness_threshold = 0.12
The median_mesh_center was requested and HAS already been calculated
functions_list = [<function median_mesh_center at 0x7f46ba9210d0>, <function n_faces_branch at 0x7f46ba920b70>]
segment_name before cgal = L0_0



segment_name before cgal = L0_1



segment_name before cgal = L0_2



Removed 1 spines because of side length greater than 5000
segment_name before cgal = L0_3



Removed 1 spines because of side length greater than 5000
segment_name before cgal = L0_4


...


segment_name before cgal = L5_0



segment_name before cgal = L5_1



Removed 1 spines because of side length greater than 5000
segment_name before cgal = L5_2



segment_name before cgal = L6_0



Removed 1 spines because of side length greater than 5000
segment_name before cgal = L7_0


In [15]:
nviz.plot_spines(exc_obj)


 Working on visualization type: mesh

 Working on visualization type: skeleton


In [16]:
ret_col = nviz.visualize_neuron(exc_obj,
                      visualize_type=["mesh"],
                     limb_branch_dict="all",
                     mesh_resolution="limb",
                     return_color_dict=True)


 Working on visualization type: mesh


In [17]:
import neuron_searching as ns

In [23]:
ax_limb_branch = ns.axon_width_like_segments(exc_obj,include_ais=True)
ax_limb_branch

current_query = ((n_spines < 6) and ((median_mesh_center < 600)) and spine_density <= 0.00007)
functions_list = [<function median_mesh_center at 0x7f469af4d0d0>, <function n_spines at 0x7f4698021c80>, <function spine_density at 0x7f469af4d400>]


{'L0': array([10, 16]),
 'L1': array([ 1,  2,  3,  4,  7,  8, 11, 13, 14, 15]),
 'L2': array([0, 2, 4]),
 'L4': array([2]),
 'L5': array([0, 2]),
 'L7': array([0])}

In [24]:
nviz.visualize_neuron(exc_obj,
                      visualize_type=["mesh"],
                     limb_branch_dict=ax_limb_branch,
                     mesh_color="red",
                     mesh_whole_neuron=True)


 Working on visualization type: mesh


In [29]:
import neuron_utils as nru

In [33]:
nru.viable_axon_limbs_by_starting_angle(exc_obj,
                                           verbose=True,
                                            above_threshold = False,
                                           soma_angle_threshold = 45)

functions_list = [<function soma_starting_angle at 0x7f469af4c0d0>]
Limb L0 soma angle: 31.96 
Limb L1 soma angle: 120.57 
Limb L2 soma angle: 112.62 
Limb L3 soma angle: 134.92 
Limb L4 soma angle: 100.5 
Limb L5 soma angle: 120.96 
Limb L6 soma angle: 126.82 
Limb L7 soma angle: 151.97 


[0]

In [25]:
import classification_utils as clu

In [28]:
clu.axon_classification(exc_obj,
                       plot_axons=True,
                       plot_axon_errors=True)

current_query = ((n_spines < 6) and ((median_mesh_center < 600)) and spine_density <= 0.00007)
functions_list = [<function median_mesh_center at 0x7f469af4d0d0>, <function n_spines at 0x7f4698021c80>, <function spine_density at 0x7f469af4d400>]
functions_list = [<function axon_segment at 0x7f469af4dea0>]

Part 1: Axon like branchese 
{'L0': array([10, 16]), 'L1': array([ 1,  2,  3,  4,  7,  8, 11, 13]), 'L2': array([0, 2]), 'L5': array([0]), 'L7': array([0])}
functions_list = [<function soma_starting_angle at 0x7f469af4c0d0>]
Limb L0 soma angle: 31.96 
Limb L1 soma angle: 120.57 
Limb L2 soma angle: 112.62 
Limb L3 soma angle: 134.92 
Limb L4 soma angle: 100.5 
Limb L5 soma angle: 120.96 
Limb L6 soma angle: 126.82 
Limb L7 soma angle: 151.97 

Part 2: possible_axon_limbs = [1, 2, 3, 4, 5, 6, 7]
functions_list = [<function skeletal_distance_from_soma at 0x7f469af4db70>]
functions_list = [<function skeletal_distance_from_soma at 0x7f469af4db70>]

Working on Limb L1
nodes_to_eliminate = 

current_query = ((n_spines < 6) and ((median_mesh_center < 200)) and spine_density <= 0.00007)
functions_list = [<function median_mesh_center at 0x7f469af4d0d0>, <function n_spines at 0x7f4698021c80>, <function spine_density at 0x7f469af4d400>]
functions_list = [<function axon_segment at 0x7f469af4dea0>]
functions_list = [<function labels_restriction at 0x7f469af4d510>, <function median_mesh_center at 0x7f469af4d0d0>]

 Working on visualization type: mesh


({'L7': array([0.])},
 {'L0': array([16]), 'L1': array([2, 4, 8]), 'L2': array([0, 2])})

# How to get a number to classify the sparsely spiny neurons from fully spiney

In [ ]:
messy_inh = (minnie.Decomposition & dict(segment_id=864691135510477264)).fetch1("decomposition")
messy_inh.calculate_spines()

In [ ]:
exc_obj_2 =  (minnie.Decomposition & dict(segment_id=864691135847929438)).fetch1("decomposition")
exc_obj_2.calculate_spines()

In [ ]:
nviz.plot_spines(exc_obj_2)

In [ ]:
ns = reload(ns)

In [ ]:
curr_neuron_obj  = exc_obj#sparsely_spined_excitatory#messy_inh
curr_neuron_obj = sparsely_spined_excitatory
curr_neuron_obj = exc_obj
curr_neurong_obj = exc_obj_2

lower_width_bound = 140
upper_width_bound = 380
spine_threshold = 2
skeletal_distance_threshold = 30000
skeletal_length_threshold = 10000
min_branches_to_process = 4

return_dataframe = False

close_limb_branch_dict = ns.query_neuron(curr_neuron_obj,
                                            functions_list=["skeletal_distance_from_soma_excluding_node","no_spine_median_mesh_center",
                                                            "n_spines","spine_density","skeletal_length"],
                                            query=(f"(skeletal_distance_from_soma_excluding_node<{skeletal_distance_threshold})"
                                                   f" and (no_spine_median_mesh_center > {lower_width_bound})"
                                                   f" and (no_spine_median_mesh_center < {upper_width_bound})"
                                                  f" and (n_spines > {spine_threshold})"
                                                   f" and skeletal_length > {skeletal_length_threshold} "
                                                  ),
                                             return_dataframe=return_dataframe


                                          )

if return_dataframe is False:
    nviz.visualize_neuron(curr_neuron_obj,
                          visualize_type=["mesh"],
                         limb_branch_dict=close_limb_branch_dict,
                          mesh_color="red",
                          mesh_whole_neuron=True)
else:
    print(f"Number of branches = {len(close_limb_branch_dict)}")
    if len(close_limb_branch_dict)>min_branches_to_process:                          
        print(f'median spine density = {np.median(close_limb_branch_dict["spine_density"].to_numpy())}')
    else:
        print(f"Not enough branches to process: only {len(close_limb_branch_dict)} and threshold set as {min_branches_to_process}")

In [ ]:
ret_col = nviz.visualize_neuron(curr_neuron_obj,
                     limb_branch_dict=dict(L0="all"),
                     return_color_dict=True)

In [ ]:
nviz.plot_spines(inh_obj)

In [ ]:
ret_color_limb = nviz.visualize_neuron(sparsely_spined_excitatory,
                      visualize_type=["mesh"],
                     limb_branch_dict="all",
                     mesh_resolution="limb",
                     return_color_dict=True)

In [ ]:
nviz.plot_soma_limb_concept_network(sparsely_spined_excitatory,
                                   node_colors=ret_color_limb)

In [ ]:
ret_col_branches = nviz.visualize_neuron(sparsely_spined_excitatory,
                     limb_branch_dict=dict(L2="all"),
                     return_color_dict=True)

In [ ]:
import spine_utils as spu

In [ ]:
curr_spines = spu.get_spine_meshes_unfiltered_from_mesh(curr_branch.mesh)

In [ ]:
curr_branch = sparsely_spined_excitatory[2][3]
nviz.plot_branches_with_spines(curr_branch)

In [ ]:
import neuron_searching as ns

In [ ]:
axon_like_limb_branch_dict = ns.axon_width_like_segments(sparsely_spined_excitatory)
axon_like_limb_branch_dict

In [ ]:
nviz.visualize_neuron(sparsely_spined_excitatory,
                      visualize_type=["mesh"],
                     limb_branch_dict=axon_like_limb_branch_dict,
                     mesh_color="red",
                     mesh_whole_neuron=True)

# Redefining the characteristics of an axon finding

In [ ]:
curr_neuron_obj = exc_obj
axon_like_limb_branch_dict = ns.axon_width_like_segments(curr_neuron_obj,
                                                        include_ais=True)

nviz.visualize_neuron(curr_neuron_obj,
                      visualize_type=["mesh"],
                     limb_branch_dict=axon_like_limb_branch_dict,
                     mesh_color="red",
                      mesh_color_alpha=1,
                     mesh_whole_neuron=True)

In [ ]:
current_functions_list = ["axon_segment"]
final_axon_like_classification = ns.query_neuron(curr_neuron_obj,

                                   query="axon_segment==True",
                                   function_kwargs=dict(limb_branch_dict =axon_like_limb_branch_dict,
                                                        downstream_face_threshold=downstream_face_threshold,
                                                        width_match_threshold=width_match_threshold,
                                                       print_flag=False),
                                   functions_list=current_functions_list)

if plot_axon_like_segments:
    nviz.visualize_neuron(curr_neuron_obj,
                          visualize_type=["mesh"],
                         limb_branch_dict=final_axon_like_classification,
                         mesh_color="red",
                          mesh_color_alpha=1,
                         mesh_whole_neuron=True)

In [ ]:
ret_col = nviz.visualize_neuron(exc_obj,
                     limb_branch_dict=dict(L0="all"),
                     return_color_dict=True)

In [ ]:
nviz.plot_limb_concept_network_2D(exc_obj,
                                  node_colors=ret_col,
                                 starting_soma_group=1)

In [ ]:
"""
good spine: 66117646.61667633


"""

In [ ]:
tu.mesh_volume(exc_obj[0][65].spines[0])

In [ ]:
nviz.plot_objects(curr_branch.spines[2])

In [ ]:
66117646.61667633*0.3

In [ ]:
np.array(curr_branch.spines_volume)/ 66117646.61667633

In [ ]:
curr_branch.spines_volume

In [ ]:
curr_branch = exc_obj[0][27]
nviz.plot_branches_with_spines(curr_branch)

In [ ]:
curr_branch = exc_obj[0][5]

In [ ]:
import skeleton_utils as sk

In [ ]:
import trimesh_utils as tu
leftover_spines = tu.filter_meshes_by_containing_coordinates(curr_branch.spines,
                                        sk.find_branch_endpoints(curr_branch.skeleton),
                                                filter_away=True,
                                           method="distance",
                                           distance_threshold=1500,
                                           verbose=False,
                                           return_indices=False)

In [ ]:
curr_branch.spines

In [ ]:
nviz.plot_branches_with_spines(exc_obj[0][5])

In [ ]:
plot_branches_with_spines

In [ ]:
nviz.plot_spines(exc_obj)

In [ ]:
curr_neuron_obj = exc_obj
axon_like_limb_branch_dict = ns.axon_width_like_segments(curr_neuron_obj,
                                                        include_ais=True)

nviz.visualize_neuron(curr_neuron_obj,
                          visualize_type=["mesh"],
                         limb_branch_dict=axon_like_limb_branch_dict,
                         mesh_color="red",
                          mesh_color_alpha=1,
                         mesh_whole_neuron=True)
)

In [ ]:
current_functions_list = ["axon_segment"]
final_axon_like_classification = ns.query_neuron(curr_neuron_obj,

                                   query="axon_segment==True",
                                   function_kwargs=dict(limb_branch_dict =axon_like_limb_branch_dict,
                                                        downstream_face_threshold=5000,
                                                        width_match_threshold=50,
                                                       print_flag=False),
                                   functions_list=current_functions_list)


nviz.visualize_neuron(curr_neuron_obj,
                      visualize_type=["mesh"],
                     limb_branch_dict=final_axon_like_classification,
                     mesh_color="red",
                      mesh_color_alpha=1,
                     mesh_whole_neuron=True)